In [3]:
import torch
import torch.nn as nn
import os
import numpy as np
import scipy.sparse as sp

In [16]:
import sys
class Dataset:
    def __init__(self, path, train=True):
        file = path + 'train.txt' if train else path + 'test.txt'
        self.num_user = 0
        self.num_item = 0
        self.users = []
        self.unique_users = []
        self.items = []
        with open(file) as f:
            for line in f.readlines():
                if len(line) > 0:
                    line = line.strip('\n').split(' ')
                    userId = int(line[0])
                    item = [int(id) for id in line[1:]]
                    self.num_user = max(self.num_user, userId)
                    self.num_item = max(self.num_item, max(item))
                    self.unique_users.append(userId)
                    self.items.extend(item)
                    self.users.extend([userId] * len(item))
        self.users = np.array(self.users)
        self.unique_users = np.array(self.unique_users)
        self.items = np.array(self.items)
        self.num_user += 1
        self.num_item += 1

        self.userItemMatrix = sp.csr_matrix((np.ones(len(self.users)), 
                                            (self.users, self.items)), 
                                            shape=(self.num_user,self.num_item))
        
        self.Graph = None
        self.build_graph()
    def _sp_to_torch(self,x):
        coo = x.tocoo().astype(np.float32)
        row = torch.LongTensor(coo.row)
        col = torch.LongTensor(coo.col)
        ind = torch.stack([row,col])
        data = torch.FloatTensor(coo.data)
        return torch.sparse.FloatTensor(ind,data, torch.Size(coo.shape))
    def build_graph(self):

        R = self.userItemMatrix.tocsr().astype(np.float32)
        adj_mat = sp.bmat([[None, R], [R.T.tocsr().astype(np.float32), None]], format="csr", dtype=np.float32)


        rowsum = np.array(adj_mat.sum(axis=1))
        d_inv = np.power(rowsum, -0.5).flatten()
        d_inv[np.isinf(d_inv)] = 0.
        d_mat = sp.diags(d_inv)

        norm_adj = d_mat.dot(adj_mat)
        norm_adj = norm_adj.dot(adj_mat)
        norm_adj = norm_adj.tocsr()

        self.Graph = self._sp_to_torch(norm_adj)
        self.Graph = self.Graph.coalesce()
    def get_graph(self):
        return self.Graph
    def get_user_number(self):
        return self.num_user
    def get_item_number(self):
        return self.num_item
    def get_positive_items(self, users):
        positive_items = []

        for user in users:
            pos_item = self.userItemMatrix[user].nonzero()[1]
            positive_items.append(pos_item)

        return positive_items

In [17]:
amazon_books = Dataset('./data/yelp2018/')

/tmp/ipykernel_4825/2935478859.py:39: UserWarning: torch.sparse.SparseTensor(indices, values, shape, *, device=) is deprecated.  Please use torch.sparse_coo_tensor(indices, values, shape, dtype=, device=). (Triggered internally at ../torch/csrc/utils/tensor_new.cpp:605.)
  return torch.sparse.FloatTensor(ind,data, torch.Size(coo.shape))


In [18]:
amazon_books.get_positive_items([2,3])

[array([553, 554, 555, 556, 557, 558, 559, 560, 561, 562, 563, 564, 565,
        566, 567, 568, 569, 570, 571, 572, 573, 574, 575, 576, 577, 578,
        579, 580, 581, 582, 583, 584, 585, 586, 587, 588, 589, 590, 591,
        592, 593, 594, 595, 596, 597, 598, 599, 600, 601, 602, 603, 604,
        605, 606, 607, 608, 609, 610, 611, 612, 613, 614, 615, 616, 617,
        618, 619, 620], dtype=int32),
 array([553, 554, 558, 560, 562, 565, 568, 572, 577, 580, 587, 590, 599,
        621, 622, 623], dtype=int32)]

In [36]:
import torch.nn.functional  as F
class LightGCN(nn.Module):
    def __init__(self, dataset: Dataset, config):
        super().__init__()
        self.dataset = dataset
        self.num_user = dataset.get_user_number()
        self.num_item = dataset.get_item_number()
        self.graph = dataset.get_graph()
        self.embedding_dim = config.embedding_dim
        self.lr = config.lr
        self.layers = config.layers
        self.embedding_user = nn.Embedding(
            num_embeddings=self.num_user,
            embedding_dim=self.embedding_dim)
        self.embedding_item = nn.Embedding(
            num_embeddings=self.num_item,
            embedding_dim=self.embedding_dim)
        nn.init.normal_(self.embedding_user.weight)
        nn.init.normal_(self.embedding_item.weight)

    def propagate(self):
        user_weight = self.embedding_user.weight
        item_weight = self.embedding_item.weight
        embed = torch.cat([user_weight, item_weight])
        
        embs = [embed]
        for layer in range(self.layers):
            embed = torch.sparse.mm(self.graph, embed)
            embs.append(embed)
        embs = torch.stack(embs, dim=1)
        out = torch.mean(embs, dim=1)
        users, items = torch.split(out, [self.num_user, self.num_item])
        return users, items
    def forward(self, users, items):
        user_emb, item_emb = self.propagate()

        user_emb = user_emb[users]
        item_emb = item_emb[items]

        scores = torch.mul(user_emb, item_emb).sum(dim=1)
        return scores
    def bpr_loss(self, users):
        pos_items = self.dataset.get_positive_items(users)
        user_embs, item_embs = self.propagate()
        loss = 0.0
        for index, user in enumerate(users):
            user_emb = user_embs[user]
            pos_item = pos_items[index]
            pos_embs = item_embs[pos_item]
            neg_item = ~np.isin(range(self.num_item), pos_item)
            neg_embs = item_embs[neg_item]
            yui = torch.mul(user_emb, pos_embs).sum(axis=1)
            yuj = torch.mul(user_emb, neg_embs).sum(axis=1, keepdim=True)
            loss += F.softplus(yuj - yui).sum()
        return loss

In [29]:
class Config:
    def __init__(self):
        self.embedding_dim = 128
        self.lr = 0.01
        self.layers = 3
config = Config()

In [37]:
lgcn = LightGCN(amazon_books, config)

In [39]:
loss=lgcn.bpr_loss([2,3])

In [40]:
loss.backward()